In [10]:


import pathlib
import sqlite3

import geopandas as gpd

from utils import fetch_handbook_dataframe

data_dir = pathlib.Path("..").resolve() / "data"
gpkg = str(data_dir / "aped.gpkg")
# Create connection and load spatialite extension
df = gpd.read_file(gpkg, layer="minister")

aph_df = fetch_handbook_dataframe(df)
aph_df

,PHID,GivenName,MiddleNames,FamilyName,PreferredName,DisplayName,DateOfBirth,DateOfDeath,PlaceOfBirth,PlaceOfDeath,...,RepresentedMinistries,RepresentedShadowMinistries,ParliamentaryPositions,Honours,Occupations,SecondaryOccupations,Qualifications,ElectorateService,PartyParliamentaryService,PartyCommitteeService
0,300147,Aaron,,VIOLI,,"VIOLI, Aaron",,,,,...,[],[],[],[],[],[],[],"[{'Electorate': 'Casey', 'State': 'Victoria', ...","[{'RoSId': 59490, 'RoSType': 'Parliamentary Se...","[{'RoSId': 60048, 'RoSType': 'Committee Servic..."
0,M3C,Adam,Paul,BANDT,,"BANDT, Dr Adam Paul",1972-03-11,,Adelaide,,...,[],[],[],[],"[Barrister from 2008 to 2010., Research office...","[Solicitor, Social Professionals nfd, Barriste...","[Doctor of Philosophy, Monash University, Bach...","[{'Electorate': 'Melbourne', 'State': 'Victori...","[{'RoSId': 27172, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1164, 'RoSType': 'Committee Service..."
0,269375,Alexander,Charles,ANTIC,(Alex),"ANTIC, Alexander (Alex) Charles",1974-12-17,,Adelaide,,...,[],[],[],[],[Solicitor from 2002 to 2019.],"[Solicitor, Legal, Social and Welfare Professi...","[Bachelor of Laws, University of Adelaide, Bac...",[],"[{'RoSId': 54265, 'RoSType': 'Parliamentary Se...","[{'RoSId': 54358, 'RoSType': 'Committee Servic..."
0,HWO,Alexander,George,HAWKE,(Alex),"HAWKE, the Hon. Alexander (Alex) George",1977-07-09,,Wollongong,,...,"[Assistant Minister, Assistant Minister, Assis...",[],[Member of the Speaker's Panel],[],"[Adviser to R Williams, MLA, 2007., Adviser to...","[Retail Manager (General), Hospitality, Retail...","[Bachelor of Arts (Government and Philosophy),...","[{'Electorate': 'Mitchell', 'State': 'New Sout...","[{'RoSId': 27382, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1508, 'RoSType': 'Committee Service..."
0,144732,Alicia,Emma,PAYNE,,"PAYNE, Alicia Emma",1982-07-24,,Canberra,,...,[],[],[],[],"[Chief of Staff to Hon. J Macklin MP, Shadow M...","[Policy Analyst, Business, Human Resource and ...","[Bachelor of Economics Honours, University of ...","[{'Electorate': 'Canberra', 'State': 'Australi...","[{'RoSId': 54199, 'RoSType': 'Parliamentary Se...","[{'RoSId': 54738, 'RoSType': 'Committee Servic..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,288713,Sam,,BIRRELL,,"BIRRELL, Sam",,,,,...,[],[],[],[],[],[],[],"[{'Electorate': 'Nicholls', 'State': 'Victoria...","[{'RoSId': 59510, 'RoSType': 'Parliamentary Se...","[{'RoSId': 60028, 'RoSType': 'Committee Servic..."
0,300130,Sam,,LIM,,"LIM, Sam",1961-06-06,,"Parit Zing, Muar, Johor",,...,[],[],[],"[Police Officer of the Year, 2020.]","[Police Officer from 6.11.2006 to 20.5.2022., ...","[Security Officers and Guards nec, Police Offi...","[Diploma in Public Safety (Policing), Western ...","[{'Electorate': 'Tangney', 'State': 'Western A...","[{'RoSId': 59223, 'RoSType': 'Parliamentary Se...","[{'RoSId': 59842, 'RoSType': 'Committee Servic..."
0,300122,Samuel,Thomas,RAE,(Sam),"RAE, Samuel (Sam) Thomas",1986-09-26,,Canberra,,...,[],[],[],[],"[Partner at PwC from 2019 to 2021., State Secr...","[Call or Contact Centre Operator, Child Care W...","[Master of Business Administration, Melbourne ...","[{'Electorate': 'Hawke', 'State': 'Victoria', ...","[{'RoSId': 59317, 'RoSType': 'Parliamentary Se...","[{'RoSId': 59813, 'RoSType': 'Committee Servic..."
0,I0U,Sarah,Coral,HANSON-YOUNG,,"HANSON-YOUNG, Sarah Coral",1981-12-23,,Melbourne,,...,[],[],[],[],"[Freelance writer and columnist from 2007., Ba...","[Bank Worker, Sales Assistant (General), Journ...","[Bachelor of Social Science, University of Ade...",[],"[{'RoSId': 27831, 'RoSType': 'Parliamentary Se...","[{'RoSId': 3191, 'RoSType': 'Committee Service..."


In [11]:
# Tidu up and create sub tables
ministers_table = aph_df[
    ["PHID", "GivenName", "MiddleNames", "FamilyName", "PreferredName", "DisplayName", "DateOfBirth", "PlaceOfBirth",
     "Image", "Gender", "MaritalStatus", "CountryOfBirth", "StateOfBirth", "Party", "PartyAbbrev", "SenateState",
     "State", "StateAbbrev", "InCurrentParliament", "ServiceHistory_Start", "ServiceHistory_End", "ServiceHistory_Days",
     "ElectedMemberNo", "ElectedSenatorNo", "FirstNations"]]
ministers_occupations = aph_df[["PHID", "Occupations"]].explode("Occupations")
ministers_secondary_occupations = aph_df[["PHID", "SecondaryOccupations"]].explode("SecondaryOccupations")
ministers_qualifications = aph_df[["PHID", "Qualifications"]].explode("Qualifications")
ministers_secondary_schools = aph_df[["PHID", "SecondarySchool"]].explode("SecondarySchool")

In [22]:
mss = ministers_secondary_schools.set_index("PHID")
mss = mss.SecondarySchool.str.split("/").explode()
mss = mss.reset_index()
db_con = sqlite3.connect(gpkg)

mss.dropna().to_sql("ministers_secondary_school", db_con, if_exists="replace",
                                         index_label="fid")

231

In [10]:
# Write data to gpkg
db_con = sqlite3.connect(gpkg)
ministers_table.to_sql("ministers_aph", db_con, if_exists="replace")
ministers_occupations.dropna().to_sql("ministers_occupations", db_con, if_exists="replace", index_label="fid")
ministers_secondary_occupations.dropna().to_sql("ministers_secondary_occupations", db_con, if_exists="replace",
                                                index_label="fid")
ministers_qualifications.dropna().to_sql("ministers_secondary_qualifications", db_con, if_exists="replace",
                                         index_label="fid")
aph_df.to_parquet(data_dir / "aph_all.parquet")

361

In [23]:
import pandas as pd

import pathlib
import geopandas as gpd

data_dir = pathlib.Path("..").resolve() / "data"
gpkg = str(data_dir / "aped.gpkg")

aus_schools = gpd.read_file(data_dir / "school-location-2022.csv")
aus_schools["geometry"] = gpd.points_from_xy(aus_schools.Longitude, aus_schools.Latitude)
aus_schools.set_crs("EPSG:4326", inplace=True)
df = gpd.read_file(gpkg, layer="education")
df.set_crs("EPSG:4326", inplace=True)
aus_schools = aus_schools[aus_schools["School Type"].isin(["Secondary", "Combined"])]

In [34]:
merged = pd.merge(df, aus_schools, how="inner", left_on=df.school_name, right_on=aus_schools["School Name"])
matched_schools = merged[merged.geometry_x.buffer(0.01).intersects(merged.geometry_y)]
cleaned_matched_schools = matched_schools[
    ["school_name", "school_link", "operational_status", "ACARA SML ID", "Suburb", "State", "Postcode", "School Sector",
     "School Type", "ABS Remoteness Area", "ABS Remoteness Area Name", "Meshblock", "Statistical Area 1",
     "Statistical Area 2", "Statistical Area 2 Name", "Statistical Area 3", "Statistical Area 3 Name",
     "Statistical Area 4", "Statistical Area 4 Name", "Local Government Area", "Local Government Area Name",
     "State Electoral Division", "State Electoral Division Name", "Commonwealth Electoral Division",
     "Commonwealth Electoral Division Name", "geometry_x"]]
cleaned_matched_schools.set_geometry("geometry_x", inplace=True)
cleaned_matched_schools.rename_geometry("geometry", inplace=True)
cleaned_matched_schools.to_file(str(data_dir / "aped.gpkg"), layer='secondary_schools', driver="GPKG", )

/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.rename(columns={geometry_col: col}, inplace=inplace)
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.p

In [8]:
aus_schools_reproj = aus_schools.to_crs("EPSG:9473")
df_repoj = df[df.is_high_school > 0].to_crs("EPSG:9473")
matched_dist = df_repoj.sjoin_nearest(aus_schools_reproj, how="left", max_distance=1000)
merged = pd.merge(df, aus_schools, how="inner", left_on=df.school_name, right_on=aus_schools["School Name"])
merged.set_crs("EPSG:4326").to_crs("EPSG:9473")

In [26]:
matched_dist

,ogc_fid,school_name,school_link,is_university,is_high_school,is_alt_edu,operational_status,wkt,is_public,geometry,...,Statistical Area 3,Statistical Area 3 Name,Statistical Area 4,Statistical Area 4 Name,Local Government Area,Local Government Area Name,State Electoral Division,State Electoral Division Name,Commonwealth Electoral Division,Commonwealth Electoral Division Name
1,2.0,"Mazenod College, Perth",http://www.wikidata.org/entity/Q1914698,0,1,0,NaN,Point(116.039 -32.0067),NaN,POINT (-1490830.304 -3581328.902),...,50605,Kalamunda,506,Perth - South East,54200,Kalamunda,52702,Kalamunda (East Metropolitan),509,Hasluck
3,4.0,Anglican Church Grammar School,http://www.wikidata.org/entity/Q4763439,0,1,0,NaN,Point(153.054 -27.4819),NaN,POINT (2045962.495 -3149848.066),...,30502,Brisbane Inner - East,305,Brisbane Inner City,31000,Brisbane,30076,South Brisbane,313,Griffith
4,5.0,Aquinas College,http://www.wikidata.org/entity/Q4782971,0,1,0,NaN,Point(145.235555555 -37.823055555),NaN,POINT (1170372.722 -4195626.036),...,21103,Maroondah,211,Melbourne - Outer East,24410,Maroondah,27101,Ringwood (Eastern Metropolitan),211,Deakin
5,6.0,Armadale Senior High School,http://www.wikidata.org/entity/Q4792525,0,1,0,NaN,Point(116.015 -32.1617),NaN,POINT (-1490896.410 -3598813.109),...,50601,Armadale,506,Perth - South East,50210,Armadale,50202,Armadale (East Metropolitan),502,Burt
5,6.0,Armadale Senior High School,http://www.wikidata.org/entity/Q4792525,0,1,0,NaN,Point(116.015 -32.1617),NaN,POINT (-1490896.410 -3598813.109),...,50601,Armadale,506,Perth - South East,50210,Armadale,50202,Armadale (East Metropolitan),502,Burt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,198.0,Victorian School of Correspondence,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(143.3906074 -36.9847807),NaN,POINT (1015944.306 -4087658.653),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,217.0,Westfields Sports High School,http://www.wikidata.org/entity/Q7988785,0,1,0,NaN,Point(150.9199898 -33.867973),NaN,POINT (1734669.468 -3824698.788),...,12702,Fairfield,127,Sydney - South West,12850,Fairfield,10029,Fairfield,128,McMahon
278,183.0,Wilson Park Secondary College,https://www.wikidata.org/wiki/Q7604726,0,1,0,amalgamated,Point(144.574444 -37.708333),NaN,POINT (1113350.803 -4177123.446),...,21304,Melton - Bacchus Marsh,213,Melbourne - West,24650,Melton,24808,Melton (Western Victoria),218,Gorton
279,186.0,Wynyard High School,http://www.wikidata.org/entity/Q916063,0,1,0,NaN,Point(145.7169952 -40.9868049),NaN,POINT (1175621.137 -4544906.794),...,60401,Burnie - Ulverstone,604,West and North West,65410,Waratah-Wynyard,60209,Braddon (Murchison),602,Braddon


In [9]:
from thefuzz import fuzz


def calc_dist(x):
    if isinstance(x["School Name"], str) and isinstance(x["school_name"], str):
        s1, s2 = x["school_name"], x["School Name"]
        return fuzz.ratio(s1, s2)
    return 0


matched_dist["school_name_str_match"] = matched_dist.apply(calc_dist, axis=1)
merged["school_name_str_match"] = merged.apply(calc_dist, axis=1)

In [10]:
all_merged = pd.concat([matched_dist, merged])

/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as GDA2020 / Australian Albers (the single non-null crs provided).
  warnings.warn(
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(


In [11]:
all_merged_deduped =  all_merged.sort_values("school_name_str_match").drop_duplicates("school_name", keep="last")

In [13]:
all_merged_deduped[all_merged_deduped["ACARA SML ID"].isna()]

,ogc_fid,school_name,school_link,is_university,is_high_school,is_alt_edu,operational_status,wkt,is_public,geometry,...,Local Government Area,Local Government Area Name,State Electoral Division,State Electoral Division Name,Commonwealth Electoral Division,Commonwealth Electoral Division Name,school_name_str_match,key_0,geometry_x,geometry_y
232,219.0,Harvey Agricultural Senior High School,NaN,0,1,0,NaN,Point(115.8772958 -33.1340183),NaN,POINT (-1489993.022 -3708241.967),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
230,201.0,Greythorn High School,http://www.wikidata.org/entity/Q5608510,0,1,0,NaN,Point(145.096 -37.8017),1.0,POINT (1158316.899 -4192010.732),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
203,NaN,Boronia Heights College,http://www.wikidata.org/entity/Q4946270,0,1,0,amalgamated,Point(145.300833 -37.86),1.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
138,141.0,St. Paul's School,http://www.wikidata.org/entity/Q7595174,0,1,0,NaN,Point(121.52749 25.02591),NaN,POINT (-1453385.237 2238181.201),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
267,231.0,St Andrew Muar Johor,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(102.5729627 2.05415),NaN,POINT (-3575232.595 -205975.521),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
133,136.0,Norwood High School,http://www.wikidata.org/entity/Q62516299,0,1,0,NaN,Point(144.961 -37.8014),NaN,POINT (17865737.514 -9060438.022),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
271,190.0,Swanbourne High School,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(115.76678 -31.9711404),NaN,POINT (-1516625.221 -3580586.069),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
78,81.0,Enfield High School,http://www.wikidata.org/entity/Q5377304,0,1,0,closed,Point(138.6044383 -34.849003347396675),NaN,POINT (601825.541 -3820876.028),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
276,198.0,Victorian School of Correspondence,http://www.wikidata.org/entity/None,0,1,0,NaN,Point(143.3906074 -36.9847807),NaN,POINT (1015944.306 -4087658.653),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None
237,244.0,Kambalda Senior High School,NaN,0,1,0,NaN,Point(121.662514 -31.201572),NaN,POINT (-974312.928 -3437397.597),...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,None


In [12]:
all_merged_deduped = all_merged_deduped[
    ["school_name", "school_link", "operational_status", "ACARA SML ID", "Suburb", "State", "Postcode", "School Sector",
     "School Type", "ABS Remoteness Area", "ABS Remoteness Area Name", "Meshblock", "Statistical Area 1",
     "Statistical Area 2", "Statistical Area 2 Name", "Statistical Area 3", "Statistical Area 3 Name",
     "Statistical Area 4", "Statistical Area 4 Name", "Local Government Area", "Local Government Area Name",
     "State Electoral Division", "State Electoral Division Name", "Commonwealth Electoral Division",
     "Commonwealth Electoral Division Name", "geometry_x"]]
all_merged_deduped.set_geometry("geometry_x", inplace=True)
all_merged_deduped.rename_geometry("geometry", inplace=True)
all_merged_deduped.to_crs("EPSG:4326", inplace=True)


In [22]:
all_merged_deduped.to_file(gpkg, layer='education__acara', driver="GPKG")

In [2]:
# fetch missing aph members data

import pathlib
import sqlite3

import geopandas as gpd

from utils import fetch_handbook_dataframe
import sqlite3
import pathlib
import pandas as pd
import geopandas as gpd

root_data_dir = pathlib.Path("../..").resolve() / "data"
ext_data_dir =root_data_dir / "external"

gpkg = root_data_dir / "aped.gpkg"
db_con = sqlite3.connect(gpkg)

ministers = pd.read_sql("SELECT * from vw_ministers_aph WHERE PHID IS NULL", db_con)
ministers

,id,member,party,group,district,is_senator,is_representative,graduated,mp_id,start,...,SenateState,State,StateAbbrev,InCurrentParliament,ServiceHistory_Start,ServiceHistory_End,ServiceHistory_Days,ElectedMemberNo,ElectedSenatorNo,FirstNations
0,193,Scott Buchholz,Coalition,http://www.wikidata.org/entity/Q1065320,Wright,0,1,1,230531,2010-08-21T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
1,194,Scott Morrison,Coalition,http://www.wikidata.org/entity/Q1065320,Cook,0,1,1,E3L,2007-11-24T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
2,195,Sharon Claydon,Australian Labor Party,http://www.wikidata.org/entity/Q216082,Newcastle,0,1,1,248181,2013-09-07T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
3,196,Shayne Neumann,Australian Labor Party,http://www.wikidata.org/entity/Q216082,Blair,0,1,1,HVO,2007-11-24T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
4,197,Simon Birmingham,Coalition,http://www.wikidata.org/entity/Q1065320,South Australia,1,0,1,H6X,2007-05-03T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
5,198,Slade Brockman,Coalition,http://www.wikidata.org/entity/Q1065320,Western Australia,1,0,1,30484,2017-08-16T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
6,199,Sophie Scamps,Independent,None,Mackellar,0,1,1,299623,2022-05-21T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
7,200,Stephen Bates,Australian Greens,http://www.wikidata.org/entity/Q781486,Brisbane,0,1,1,300246,2022-05-21T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
8,201,Stephen Jones,Australian Labor Party,http://www.wikidata.org/entity/Q216082,Whitlam,0,1,1,A9B,2016-07-02T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None
9,202,Steve Georganas,Australian Labor Party,http://www.wikidata.org/entity/Q216082,Adelaide,0,1,1,DZY,2019-05-18T00:00:00Z,...,None,None,None,None,None,None,None,None,None,None


In [3]:

aph_df = fetch_handbook_dataframe(ministers)
aph_df

,PHID,GivenName,MiddleNames,FamilyName,PreferredName,DisplayName,DateOfBirth,DateOfDeath,PlaceOfBirth,PlaceOfDeath,...,RepresentedMinistries,RepresentedShadowMinistries,ParliamentaryPositions,Honours,Occupations,SecondaryOccupations,Qualifications,ElectorateService,PartyParliamentaryService,PartyCommitteeService
0,230531,Scott,Andrew,BUCHHOLZ,,"BUCHHOLZ, the Hon. Scott Andrew",1968-03-27,,Rockhampton,,...,"[Assistant Minister, Assistant Minister]",[],"[Member of the Speaker's Panel, Member of the ...",[],"[Managing director from 1992., Agri-finance ma...","[Livestock Farm Workers nec, Finance Manager, ...",[],"[{'Electorate': 'Wright', 'State': 'Queensland...","[{'RoSId': 27204, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1229, 'RoSType': 'Committee Service..."
0,E3L,Scott,John,MORRISON,,"MORRISON, the Hon. Scott John",1968-05-13,,Sydney,,...,"[Cabinet Minister, Cabinet Minister, Cabinet M...","[Shadow Minister, Shadow Minister, Shadow Mini...",[],[],[Principal at MSAS Pty. Ltd. from 2006 to 2007...,"[Policy and Planning Manager, Chief Executive ...","[Bachelor of Science Honours, University of Ne...","[{'Electorate': 'Cook', 'State': 'New South Wa...","[{'RoSId': 27505, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1825, 'RoSType': 'Committee Service..."
0,248181,Sharon,Catherine,CLAYDON,,"CLAYDON, Sharon Catherine",1964-04-26,,Camperdown,,...,[],[],"[Deputy Speaker, Member of the Speaker's Panel]",[],"[Disability support worker, Social anthropolog...","[Aged or Disabled Carer, Social Professionals ...","[Bachelor of Arts Honours, University of Sydney]","[{'Electorate': 'Newcastle', 'State': 'New Sou...","[{'RoSId': 27293, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1321, 'RoSType': 'Committee Service..."
0,HVO,Shayne,Kenneth,NEUMANN,,"NEUMANN, the Hon. Shayne Kenneth",1961-08-26,,Ipswich,,...,"[Parliamentary Secretary, Parliamentary Secret...","[Shadow Minister, Shadow Minister, Shadow Mini...",[],[],[Lawyer from 1985 to 2007.],"[Solicitor, Legal, Social and Welfare Professi...","[Bachelor of Arts, University of Queensland, B...","[{'Electorate': 'Blair', 'State': 'Queensland'...","[{'RoSId': 27510, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1841, 'RoSType': 'Committee Service..."
0,H6X,Simon,John,BIRMINGHAM,,"BIRMINGHAM, the Hon. Simon John",1974-06-14,,Adelaide,,...,"[Parliamentary Secretary, Assistant Minister, ...","[Shadow Parliamentary Secretary, Shadow Parlia...",[],[],[Director of the Winemakers' Federation of Aus...,"[Electorate Officer, Public Relations Manager,...","[Master of Business Administration, University...",[],"[{'RoSId': 27534, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1905, 'RoSType': 'Committee Service..."
0,30484,William,Edward Slade,BROCKMAN,(Slade),"BROCKMAN, the Hon. William (Slade) Edward Slade",1970-03-27,,Manjimup,,...,[],[],"[President of the Senate, Temporary Chair of C...",[],[Campaign Director of the Australian Grains Ch...,"[Public Relations Manager, Policy and Planning...","[Bachelor of Laws, University of New England, ...",[],"[{'RoSId': 26012, 'RoSType': 'Parliamentary Se...","[{'RoSId': 708, 'RoSType': 'Committee Service'..."
0,299623,Sophie,,SCAMPS,,"SCAMPS, Sophie",1971-03-15,,Wahroonga,,...,[],[],[],[],[Health Promotion in the Southwest Sydney Loca...,"[Medical Practitioners nfd, General Practition...","[Bachelor in Medicine, Bachelor in Surgery, Un...","[{'Electorate': 'Mackellar', 'State': 'New Sou...","[{'RoSId': 59279, 'RoSType': 'Parliamentary Se...",[]
0,300246,Stephen,,BATES,,"BATES, Stephen",1992-11-23,,London,,...,[],[],[],[],[Retail Sales and Customer Service],"[Sales Assistants and Salespersons nec, Sales ...","[Bachelor of Social Science, University of Que...","[{'Electorate': 'Brisbane', 'State': 'Queensla...","[{'RoSId': 59276, 'RoSType': 'Parliamentary Se...",[]
0,A9B,Stephen,Patrick,JONES,,"JONES, the Hon. Stephen Patrick",1965-06-29,,Sydney,,...,"[Assistant Treasurer, Minister]","[Shadow Parliamentary Secretary, Shadow Assist...",[],[],[Secretary of the Communi

In [4]:
# Tidu up and create sub tables
ministers_table = aph_df[
  ["PHID", "GivenName", "MiddleNames", "FamilyName", "PreferredName", "DisplayName", "DateOfBirth", "PlaceOfBirth",
   "Image", "Gender", "MaritalStatus", "CountryOfBirth", "StateOfBirth", "Party", "PartyAbbrev", "SenateState",
   "State", "StateAbbrev", "InCurrentParliament", "ServiceHistory_Start", "ServiceHistory_End", "ServiceHistory_Days",
   "ElectedMemberNo", "ElectedSenatorNo", "FirstNations"]]
ministers_occupations = aph_df[["PHID", "Occupations"]].explode("Occupations")
ministers_secondary_occupations = aph_df[["PHID", "SecondaryOccupations"]].explode("SecondaryOccupations")
ministers_qualifications = aph_df[["PHID", "Qualifications"]].explode("Qualifications")
ministers_secondary_schools = aph_df[["PHID", "SecondarySchool"]].explode("SecondarySchool")

In [6]:
ministers_table.to_sql("ministers_aph", db_con, if_exists="append", index=False)
ministers_occupations.dropna().to_sql("ministers_occupations", db_con, if_exists="append", index=False)
ministers_secondary_occupations.dropna().to_sql("ministers_secondary_occupations", db_con, if_exists="append",
                                                index=False)
ministers_qualifications.dropna().to_sql("ministers_secondary_qualifications", db_con, if_exists="append",
                                         index=False)
mss = ministers_secondary_schools.set_index("PHID")
mss = mss.SecondarySchool.str.split("/").explode()
mss = mss.reset_index()
mss.dropna().to_sql("ministers_secondary_school", db_con, if_exists="append",
                                         index=False)

NameError: name 'data_dir' is not defined

In [7]:
aph_df.to_parquet(ext_data_dir / "aph_all_extra.parquet")
